In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import scipy.interpolate as spi
import scipy.optimize as spo
import scipy.stats as sps
import altair as alt
import hysttools as ht
from pathlib import Path
from datetime import date

In [2]:
from altair_data_server import data_server
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [3]:
#repo=git.Repo('.', search_parent_directories=True)
path_root = Path(os.getcwd()).parent
data_folder = path_root / 'data'
processed_folder = data_folder / 'processed'
vsm_folder = data_folder / 'raw'/'vsm'
image_folder = data_folder / 'images'
search_pattern = re.compile('.*irmdcd$') # This is the suffix for a hysteresis file

file_list = list(filter(search_pattern.match,os.listdir(vsm_folder)))
specimens=pd.DataFrame(data={'name':file_list,'file':file_list})
specimens['name']=specimens['name'].str.replace('\.irmdcd','').str.replace('-.*','').str.lower()

location_names={'.*eg.*':'glacier','.*cb.*':'bay','.*combay.*':'bay','.*suspended.*':'suspended','.*bed.*':'bed'}
specimens['location']=specimens['name'].replace(location_names.keys(),location_names.values(),regex=True)

print(specimens)
print("\n{} IRMDCD files to process.".format(np.shape(specimens)[0]))

                       name                            file   location
0                2022cb0101               2022CB0101.irmdcd        bay
1                2022cb0103               2022CB0103.irmdcd        bay
2                2022eg0102               2022EG0102.irmdcd    glacier
3                2022eg0103             2022EG0103-2.irmdcd    glacier
4                2022eg0103               2022EG0103.irmdcd    glacier
5                2022eg0203               2022eg0203.irmdcd    glacier
6     2023pr01_bed_sediment    2023PR01_bed_sediment.irmdcd        bed
7   2023pr01_suspended_silt  2023PR01_suspended_silt.irmdcd  suspended
8     2023pr02_bed_sediment    2023PR02_bed_sediment.irmdcd        bed
9                  combay01               Combay01-2.irmdcd        bay
10                 combay01                 Combay01.irmdcd        bay

11 IRMDCD files to process.


C:\Users\paselkin\anaconda3\envs\pmagpy_env\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


In [4]:
#Hgrid=ht.make_hgrid(200,0.8,0.5)
#Hgrid2=ht.make_hgrid2(200,0.8)
#hyst_rh=pd.DataFrame(columns=['id']+["{}".format(x) for x in Hgrid])
#hyst_ih=pd.DataFrame(columns=['id']+["{}".format(x) for x in Hgrid])
s_dcd=pd.DataFrame([])
d_dcd=pd.DataFrame([])

In [5]:
file_skip=[]
hyst_stats=pd.DataFrame(columns=['id','field','remanence','gradient','Hcr','max_d2'])
all_irm=pd.DataFrame(columns=['id','field','remanence','gradient','Hcr','max_d2'])

for index,row in specimens.iterrows():
    name,file_name,location = row
    file_path = vsm_folder / file_name
    print('Working on {}\n{}\n'.format(file_name,file_path))
    if (file_name in file_skip):
        print('Skipping... {}\n'.format(name))
    if (file_name not in file_skip):
        print("Processing: {}\n".format(name))
        s,d = ht.read_dcd(file_path)
        print("IRM/Backfield file read with {} data points.".format(d.shape[0]))
        #s_dcd=pd.concat([s,s_dcd],axis=0,ignore_index=True)
        #d_dcd=pd.concat([d,d_dcd],axis=0,ignore_index=True)
        result_dcd = ht.process_dcd(s,d)
        print("Backfield processed.")
        irm_curve=d.loc[(d['type']=='irm')].copy()
        smooth = 5*(10**(2.0*np.ceil(np.log10(0.1*np.min(np.abs(irm_curve.remanence))))))
        result_irm = ht.process_irm(s,d,smooth=smooth)
        to_add=pd.Series(result_irm['irm_derivative'].to_dict())
        to_add['id']=name
        max_d2=np.max(np.gradient(pd.Series(to_add['gradient']).values,pd.Series(to_add['field']).values))
        print(">> Max IRM second derivative: {}".format(max_d2))
        to_add['max_d2']=max_d2
        to_add['Hcr']=float(result_dcd['Hcr'])
        to_add['SIRM']=float(result_irm['SIRM'])
        to_add['IRM_50mT']=float(result_irm['IRM_50mT'])
        to_add['IRM_80mT']=float(result_irm['IRM_80mT'])
        to_add['IRM_100mT']=float(result_irm['IRM_100mT'])
        to_add['IRM_300mT']=float(result_irm['IRM_300mT'])
        to_add['IRM_m80mT']=float(result_dcd['IRM_m80mT'])
        hyst_stats=hyst_stats.append(to_add,ignore_index=True)
        #if (max_d2>=2000):
        #    print("WARNING: High second derivative detected in IRM acquisition curve for specimen {}.".format(to_add['id'].first()))
        all_irm=all_irm.append(to_add,ignore_index=True)
        print("IRM data processed.")
    

Working on 2022CB0101.irmdcd
C:\Users\paselkin\Dropbox\Research\Glaciers2Bay\glacier-bay-uwt-student-projects\data\raw\vsm\2022CB0101.irmdcd

Processing: 2022cb0101

IRM/Backfield file read with 160 data points.
Backfield processed.
>> Max IRM second derivative: 596.6543637682844
IRM data processed.
Working on 2022CB0103.irmdcd
C:\Users\paselkin\Dropbox\Research\Glaciers2Bay\glacier-bay-uwt-student-projects\data\raw\vsm\2022CB0103.irmdcd

Processing: 2022cb0103

IRM/Backfield file read with 160 data points.
Backfield processed.
>> Max IRM second derivative: 2211.4004813342726
IRM data processed.
Working on 2022EG0102.irmdcd
C:\Users\paselkin\Dropbox\Research\Glaciers2Bay\glacier-bay-uwt-student-projects\data\raw\vsm\2022EG0102.irmdcd

Processing: 2022eg0102

IRM/Backfield file read with 160 data points.
Backfield processed.
>> Max IRM second derivative: 1510.5021023575518
IRM data processed.
Working on 2022EG0103-2.irmdcd
C:\Users\paselkin\Dropbox\Research\Glaciers2Bay\glacier-bay-uwt-

C:\Users\paselkin\anaconda3\envs\pmagpy_env\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


>> Max IRM second derivative: 2495.813553281472
IRM data processed.
Working on 2022eg0203.irmdcd
C:\Users\paselkin\Dropbox\Research\Glaciers2Bay\glacier-bay-uwt-student-projects\data\raw\vsm\2022eg0203.irmdcd

Processing: 2022eg0203

IRM/Backfield file read with 160 data points.
Backfield processed.
>> Max IRM second derivative: 3966.519908975536
IRM data processed.
Working on 2023PR01_bed_sediment.irmdcd
C:\Users\paselkin\Dropbox\Research\Glaciers2Bay\glacier-bay-uwt-student-projects\data\raw\vsm\2023PR01_bed_sediment.irmdcd

Processing: 2023pr01_bed_sediment

IRM/Backfield file read with 160 data points.
Backfield processed.
>> Max IRM second derivative: 574.895711400839
IRM data processed.
Working on 2023PR01_suspended_silt.irmdcd
C:\Users\paselkin\Dropbox\Research\Glaciers2Bay\glacier-bay-uwt-student-projects\data\raw\vsm\2023PR01_suspended_silt.irmdcd

Processing: 2023pr01_suspended_silt

IRM/Backfield file read with 160 data points.
Backfield processed.
>> Max IRM second derivati

In [6]:
print(all_irm.columns)
#filtered_irm=all_irm[all_irm.apply(lambda x: (pd.Series(x['gradient'])>20).any(),axis=1)]
grouped_irm=all_irm.groupby('id')
irm_max_d2s=grouped_irm['max_d2'].first()
with pd.option_context('display.max_rows', 150):
    display(irm_max_d2s.sort_values())
filtered_irm=all_irm[all_irm.apply(lambda x: (pd.Series(x['max_d2'])<100000).any(),axis=1)]
#filtered_irm=
print(filtered_irm.shape,all_irm.shape)

Index(['id', 'field', 'remanence', 'gradient', 'Hcr', 'max_d2', 'IRM_100mT',
       'IRM_300mT', 'IRM_50mT', 'IRM_80mT', 'IRM_m80mT', 'SIRM'],
      dtype='object')


id
2023pr01_bed_sediment       574.895711
2022cb0101                  596.654364
2023pr02_bed_sediment       597.677197
2023pr01_suspended_silt     857.045504
combay01                   1113.101768
2022eg0102                 1510.502102
2022cb0103                 2211.400481
2022eg0203                 3966.519909
2022eg0103                 4880.406052
Name: max_d2, dtype: float64

(11, 12) (11, 12)


In [7]:
import scipy.signal as spsig
import scipy.special as spspecial
def sgg(x, *params):
    # Skewed Generalized Gaussian (see Egli, 2004a eq. 1)
    fx=np.zeros_like(x)
    mu, sigma, p, q = params
    xprime=x-mu/sigma
    v = 1/((2.**(1.+(1./p)))*sigma*spspecial.gamma(1+(1/p)))
    m = (abs((q*np.exp(q*xprime))+((np.exp(xprime/q))/q)))/((np.exp(q*xprime))+(np.exp(xprime/q)))
    fx = v*m*np.exp(-0.5*(abs(np.log((np.exp(q*xprime))+(np.exp(xprime/q)))))**p)

    return fx

v=list(all_irm['gradient'][0].values())
f=list(all_irm['field'][0].values())
peaks,_=spsig.find_peaks(v)
display(peaks.tolist())
display(f[peaks[0]])
guess=[f[peaks[0]], 1.0, 0.1, 1.0]
popt, pcov=spo.curve_fit(sgg,f, v, guess,bounds=([0, 0, -1., 0],[np.inf, np.inf, 1., np.inf]))
vprime=list(sgg(f,popt[0],popt[1],popt[2],popt[3]))
display(popt)
print(len(v))
print(len(vprime))
display(np.array(v)-np.array(vprime))

[8, 22, 27, 35, 40, 47, 55, 64, 70, 77]

0.018437422225406452

array([0.02818905, 0.03994753, 0.99697211, 0.99999999])

81
81


array([ 3.80107987,  8.35136661,  8.78852295,  9.09785003,  9.29833856,
        9.40501989,  9.46639048,  9.5067076 ,  9.52295633,  9.51067064,
        9.4647668 ,  9.3794656 ,  9.24820508,  9.05825661,  8.76559301,
        8.31287764,  7.66363328,  6.7965945 ,  5.85833712,  5.16482288,
        4.92077708,  5.21030076,  5.72490731,  5.30273261,  3.37148788,
        1.66547804,  1.86477836,  2.73878398,  2.36045695,  0.78751661,
       -1.07161797, -2.51269203, -2.83542103, -2.09283891, -1.2600848 ,
       -1.0560788 , -1.75347226, -3.25457414, -4.39114557, -4.17484265,
       -3.5105114 , -3.64144199, -4.39317472, -5.0744518 , -5.41911686,
       -5.37198144, -5.08092008, -4.93480434, -5.07669051, -5.26728014,
       -5.33598677, -5.33015222, -5.3073917 , -5.2737646 , -5.22920179,
       -5.18389941, -5.15248785, -5.14168813, -5.15530699, -5.19767067,
       -5.27229988, -5.33231233, -5.2568159 , -5.03650376, -4.90687355,
       -5.02241097, -5.171024  , -5.12177156, -4.94476038, -4.78

In [8]:
display(all_irm)
all_irm[['id','Hcr','SIRM','IRM_50mT','IRM_80mT','IRM_m80mT','IRM_100mT','IRM_300mT','max_d2']].to_csv(processed_folder/'dcd_stats.csv')

,id,field,remanence,gradient,Hcr,max_d2,IRM_100mT,IRM_300mT,IRM_50mT,IRM_80mT,IRM_m80mT,SIRM
0,2022cb0101,"{0: 0.0, 1: 0.0125046625, 2: 0.013217877460335...","{0: 0.02854850435669612, 1: 0.1535789296789393...","{0: 9.998704508997603, 1: 14.58838302719829, 2...",0.005866,596.654364,8.819251e-06,1.011025e-05,6.448912e-06,8.143377e-06,-8.278146e-06,1.022862e-05
1,2022cb0103,"{0: 0.0, 1: 0.012505387500000001, 2: 0.0132186...","{0: 0.041716589747360706, 1: 0.144184253596404...","{0: 8.193881544977595, 1: 14.61714384120819, 2...",0.004168,2211.400481,6.440738e-06,7.568289e-06,4.672135e-06,5.954848e-06,-5.692874e-06,7.848765e-06
2,2022eg0102,"{0: 0.0, 1: 0.01250495, 2: 0.01321818135816333...","{0: 0.006635109001991987, 1: 0.118184760878228...","{0: 8.920439655995152, 1: 13.487768693963005, ...",0.014530,1510.502102,1.702561e-05,1.997928e-05,1.240168e-05,1.577970e-05,-1.592828e-05,2.048944e-05
3,2022eg0103,"{0: 0.0, 1: 0.0125067125, 2: 0.013220044383976...","{0: 0.024805450598481656, 1: 0.135757574745604...","{0: 8.871405986754924, 1: 10.956184921010902, ...",0.013778,4880.406052,1.461219e-05,1.771930e-05,1.086079e-05,1.354988e-05,-1.315304e-05,1.781252e-05
4,2022eg0103,"{0: 0.0, 1: 0.0125072625, 2: 0.013220625753733...","{0: -0.0014061622683307655, 1: 0.1075068235938...","{0: 8.707979532867373, 1: 18.286952018436523, ...",0.010034,2495.813553,1.452950e-05,1.714973e-05,1.029058e-05,1.296783e-05,-1.477793e-05,1.769390e-05
5,2022eg0203,"{0: 0.0, 1: 0.012504350000000001, 2: 0.0132175...","{0: 0.012712791802213746, 1: 0.142495447560861...","{0: 10.379000568493943, 1: 16.952168577806958,...",0.007960,3966.519909,1.485010e-05,1.730091e-05,1.138241e-05,1.382494e-05,-1.455229e-05,1.806548e-05
6,2023pr01_bed_sediment,"{0: 0.0, 1: 0.012504675000000002, 2: 0.0132178...","{0: 0.020169992733762875, 1: 0.118748850353293...","{0: 7.883360232835381, 1: 12.221182650329922, ...",0.012770,574.895711,6.445100e-06,7.703192e-06,4.483152e-06,5.897859e-06,-6.102204e-06,7.792358e-06
7,2023pr01_suspended_silt,"{0: 0.0, 1: 0.0125048625, 2: 0.013218088867520...","{0: -0.00151557965135373, 1: 0.061586399955351...","{0: 5.046195398526426, 1: 9.812673074222943, 2...",0.035330,857.045504,6.530586e-07,7.883953e-07,4.093194e-07,5.864813e-07,-5.002058e-07,8.010411e-07
8,2023pr02_bed_sediment,"{0: 0.0, 1: 0.012664417721518988, 2: 0.0133941...","{0: 0.0026671515134064246, 1: 0.07996587770802...","{0: 6.103614701786088, 1: 10.685735729855594, ...",0.028594,597.677197,1.018489e-05,1.213951e-05,6.751574e-06,9.258543e-06,-8.438245e-06,1.241570e-05
9,combay01,"{0: 0.0, 1: 0.01250985, 2: 0.01322336083418323...","{0: 0.03860491474124084, 1: 0.1555504493163195...","{0: 9.348276324262777, 1: 14.06209993334025, 2...",0.000305,1113.101768,6.963481e-06,8.364688e-06,5.154040e-06,6.564266e-06,-6.560867e-06,8.564330e-06


In [9]:
#filtered_irm=all_irm[all_irm.apply(lambda x: (pd.Series(x['gradient']).apply(lambda y: np.abs(y))>=20).any(),axis=1)].copy().reset_index(drop=True)
filtered_irm=all_irm[all_irm.apply(lambda x: (pd.Series(x['max_d2'])<2000).any(),axis=1)].copy().reset_index(drop=True)
#grouped_filtered_irm=filtered_irm.groupby('id')
#display(pd.concat(pd.DataFrame({'id':id, 'field':field,'remanence':remanence,'gradient':gradient,'max_d2':max_d2}) 
#                  for id, field, remanence, gradient, max_d2 in filtered_irm.items()))
filtered_irm=pd.merge(filtered_irm,specimens,left_on='id',right_on='name')
df_all=pd.DataFrame([])
for index,row in filtered_irm.iterrows():
    df=pd.DataFrame({'id':row['id'],'location':row['location'],'field':pd.Series(row['field']),'remanence':pd.Series(row['remanence']),
                    'gradient':pd.Series(row['gradient']),'max_d2':row['max_d2']})
    df_all=pd.concat([df_all,df],axis=0)

#print (df_all)

P=alt.Chart(df_all).mark_line(opacity=0.5).encode(
    x=alt.X('field:Q',scale=alt.Scale(type='log',domain=(0.01,1),clamp=True)),
    y=alt.Y('gradient:Q',scale=alt.Scale(domain=(0,20),clamp=True)),
    detail='id:N',
#    color='',
    tooltip=['id','field'],
    color='location:N'
).properties(
    width=300,
    height=300
).interactive()

display(P)

alt.Chart(...)

In [10]:
#filtered_irm=all_irm[all_irm.apply(lambda x: (pd.Series(x['gradient']).apply(lambda y: np.abs(y))>=20).any(),axis=1)].copy().reset_index(drop=True)
filtered_irm=all_irm[all_irm.apply(lambda x: (pd.Series(x['max_d2'])<2000).any(),axis=1)].copy().reset_index(drop=True)
#grouped_filtered_irm=filtered_irm.groupby('id')
#display(pd.concat(pd.DataFrame({'id':id, 'field':field,'remanence':remanence,'gradient':gradient,'max_d2':max_d2}) 
#                  for id, field, remanence, gradient, max_d2 in filtered_irm.items()))
filtered_irm=pd.merge(filtered_irm,specimens,left_on='id',right_on='name')
df_all=pd.DataFrame([])
for index,row in filtered_irm.iterrows():
    df=pd.DataFrame({'id':row['id'],'location':row['location'],'field':pd.Series(row['field']),'remanence':pd.Series(row['remanence']),
                    'gradient':pd.Series(row['gradient']),'max_d2':row['max_d2']})
    df_all=pd.concat([df_all,df],axis=0)

#print (df_all)

P=alt.Chart(df_all).mark_line(opacity=0.5).encode(
    x=alt.X('field:Q',scale=alt.Scale(type='log',domain=(0.01,1),clamp=True)),
    y=alt.Y('remanence:Q',scale=alt.Scale(domain=(0,1),clamp=True)),
    detail='id:N',
#    color='',
    tooltip=['id','field'],
    color='location:N'
).properties(
    width=300,
    height=300
).interactive()

display(P)

alt.Chart(...)